In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
#from tensorflow.keras.utils import to_categorical ## specific  one I wanna use
from tensorflow.keras import utils
#from tensorflow.keras.layers import Input, Dense, Dropout  # Specific ones I wanna use, can just import whole module
from tensorflow.keras import layers 
from tensorflow.keras.models import Model
import h5py
import tables
import matplotlib.pyplot as plt
#import tensorflow as tf
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve
from sklearn.utils.class_weight import compute_class_weight


In [2]:
x=1
y=3
x+y

4

In [ ]:
# Import data from all 4 collision types.

input_file_1 = "/scratch/ys20884/ml_vars/TTToHadronic/df_ml_inputs.hd5"

with h5py.File(input_file_1, "r") as f:
    #print("Contents of the file:")
    for key in f.keys():
        print(key)

df1= pd.read_hdf(input_file_1, key="df")


input_file2 = "/scratch/ys20884/ml_vars/TTToSemiLeptonic/df_ml_inputs.hd5"

with h5py.File(input_file2, "r") as f:
    #print("Contents of the file:")
    for key in f.keys():
        print(key)

df2 = pd.read_hdf(input_file2, key="df")
print("Dataframe columns:")
print(df2.columns)

input_file3 = "/scratch/ys20884/ml_vars/TTTo2L2Nu/df_ml_inputs.hd5"

with h5py.File(input_file3, "r") as f:
    #print("Contents of the file:")
    for key in f.keys():
        print(key)

df3 = pd.read_hdf(input_file3, key="df")

input_file4 = "/scratch/ys20884/ml_vars/ttH125/df_ml_inputs.hd5"

with h5py.File(input_file4, "r") as f:
    #print("Contents of the file:")
    for key in f.keys():
        print(key)

df4 = pd.read_hdf(input_file4, key="df")

print(len(df1))
print(len(df2))
print(len(df3))
print(len(df4))

df
df


In [ ]:
# make one big dataframe, full data set long to practice with
df_total = df2.append(df1)
df_total = df_total.append(df3)
df_total = df_total.append(df4)
print(len(df_total))   

# Background is 0
# Signal is 1
df_process = df_total['dataset']
df_total['dataset'] = df_total['dataset'].replace({'ttH125': 1, 'TTToSemiLeptonic': 0, 'TTToHadronic': 0, 'TTTo2L2Nu':0})
df_total.head()
# Have done test cases, is working fine.
print(len(df_total))   


In [ ]:
# drop all labels and the phi's
df_total.drop(['entry', 'BiasedDPhi', 'hashed_filename', 'weight_nominal', 'xs_weight', 
              'InputMet_phi', 'MHT_phi', 'boostedObject_phi', 'boostedTop_phi', 'boostedV_phi',
              'cleanedBJet_phi','cleanedJet_phi'], axis=1, inplace=True)#
df_total.columns            # Said BiasedDPhi looks wrong



In [ ]:
lst = []
for column in df_total:
    if df_total[column].dtype == 'object':   
        print(column, 'is a jet based variable')
        lst.append(column)
    elif column == 'dataset':    # dont want to drop dataset column (label)
        print('Need to keep labels')
        lst.append(column)
    else:
        df_total.drop([column], axis=1, inplace=True)
print(len(lst)-1, 'variables are left')    # -1 as - label
df_total.columns 

In [7]:
# Need to find the longest array in any column for my first method

# for column in df_total:
#     for i in column:
#         df_total[column]
#    x_arr = np.max(df_total[column])

In [ ]:
# loop through each column and find the maximum array length
max_lengths = {}
min_lengths = {}
for column in df_total:
    if df_total[column].dtype == 'object': 
        #print(column)
        max_lengths[column] = max([len(arr) for arr in df_total[column]])
        min_lengths[column] = min([len(arr) for arr in df_total[column]])
    else:
        pass

print('max lengths=', max_lengths)
print('min lenghts =', min_lengths)
arr_lengths = max(max_lengths.values())
print('Need to pad to', arr_lengths)

In [9]:
i=0
for column in df_total: 
    if df_total[column].dtype == 'object': 
        i+=1
        print(i)
        df_total[column] = df_total[column].apply(lambda x: np.pad(x, (0, arr_lengths-len(x)), mode='constant'))
    else:
        pass

df_total.drop(['dataset'], axis=1, inplace=True) # so only padded data, no labels    
#df_total['boostedObject_area'] = df_total['boostedObject_area'].apply(lambda x: np.pad(x, (0, arr_lengths-len(x)), mode='constant'))

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [ ]:
df_total

In [20]:
# Reshape into 3D array, instead of 2D df
padded_arr_3d = np.array([row.tolist() for row in df_total.values]).reshape(len(df_total), arr_lengths, df_total.shape[1])
# only takes like 17 seconds

In [24]:
# Save to HDF5 file
with h5py.File('padded_arr_3d.h5', 'w') as f:
    f.create_dataset('array', data=padded_arr_3d)


In [ ]:
df_total.shape

In [ ]:
df_labels = df2['dataset'].append(df1['dataset'])
df_labels = df_labels.append(df3['dataset'])
df_labels = df_labels.append(df4['dataset'])

df_labels = df_labels.replace({'ttH125': 1, 'TTToSemiLeptonic': 0, 'TTToHadronic': 0, 'TTTo2L2Nu':0})


# Save to HDF5 file
with h5py.File('4_event_types_labels.h5', 'w') as f:
    f.create_dataset('array', data=df_labels)

# This became really complicated for some reason, and isn't more efficient to a point worth working out

In [1]:
# Load padded data
#with h5py.File('padded_arr_3d.h5', 'r') as f:
 #   padded_arr_3d = f['array'][:]
#padded_arr_3d

In [7]:
df_weights = pd.concat([df1['xs_weight'], df2['xs_weight'], df3['xs_weight'], df4['xs_weight']], axis=1)
df_weights.columns = ['df1_xs_weight', 'df2_xs_weight', 'df3_xs_weight', 'df4_xs_weight']
(df_weights)

with h5py.File('4_event_types_weights.h5', 'w') as f:
    f.create_dataset('array', data=df_weights)


In [ ]:
df_labels = df2['dataset'].append(df1['dataset'])
df_labels = df_labels.append(df3['dataset'])
df_labels = df_labels.append(df4['dataset'])

df_labels = df_labels.replace({'ttH125': 1, 'TTToSemiLeptonic': 0, 'TTToHadronic': 0, 'TTTo2L2Nu':0})


# Save to HDF5 file
with h5py.File('4_event_types_labels.h5', 'w') as f:
    f.create_dataset('array', data=df_labels)




In [ ]:
model.save('trained_model.h5')

#from tensorflow.keras.models import load_model
#loaded_model = load_model('trained_model.h5')